# Create a simple tensor with random items

In [1]:
import numpy as np

# Suppress scientific notation
np.set_printoptions(suppress=True)

# Generate randomly distributed parameters
params = np.random.uniform(low=-50, high=150, size=20)

# Make sure important values are at the beginning for better debugging
params[0] = params.max() + 1
params[1] = params.min() - 1
params[2] = 0

# Round each number to the second decimal place
params = np.round(params, 2)

# Print the parameters
print(params)

[144.43 -40.1    0.     5.04  78.38 101.5   40.24  70.67  61.14  34.26
  81.59 134.05 119.69 140.25  21.37  91.09 143.43 -39.1   84.91 110.59]


# Define the quantization methods and quantize

In [2]:
# clamping func for overflow prevention
def clamp(params_q: np.array, lower_bound: int, upper_bound: int) -> np.array:
    params_q[params_q < lower_bound] = lower_bound
    params_q[params_q > upper_bound] = upper_bound
    return params_q


# asymmetric quantization method
def asymmetric_quantization(params: np.array, bits: int) -> tuple[np.array, float, int]:
    # Calculate the scale and zero point
    alpha = np.max(params)
    beta = np.min(params)
    scale = (alpha - beta) / (2**bits-1)
    zero = -1*np.round(beta / scale)
    lower_bound, upper_bound = 0, 2**bits-1
    # Quantize the parameters
    quantized = clamp(np.round(params / scale + zero), lower_bound, upper_bound).astype(np.int32)
    return quantized, scale, zero

def asymmetric_dequantize(params_q: np.array, scale: float, zero: int) -> np.array:
    return (params_q - zero) * scale

def symmetric_dequantize(params_q: np.array, scale: float) -> np.array:
    return params_q * scale

def symmetric_quantization(params: np.array, bits: int) -> tuple[np.array, float]:
    # Calculate the scale
    alpha = np.max(np.abs(params))
    scale = alpha / (2**(bits-1)-1)
    lower_bound = -2**(bits-1)
    upper_bound = 2**(bits-1)-1
    # Quantize the parameters
    quantized = clamp(np.round(params / scale), lower_bound, upper_bound).astype(np.int32)
    return quantized, scale

def quantization_error(params: np.array, params_q: np.array):
    # calculate the MSE
    return np.mean((params - params_q)**2)

(asymmetric_q, asymmetric_scale, asymmetric_zero) = asymmetric_quantization(params, 8)
(symmetric_q, symmetric_scale) = symmetric_quantization(params, 8)

print(f'Original:')
print(np.round(params, 2))
print('')
print(f'Asymmetric scale: {asymmetric_scale}, zero: {asymmetric_zero}')
print(asymmetric_q)
print('')
print(f'Symmetric scale: {symmetric_scale}')
print(symmetric_q)

Original:
[144.43 -40.1    0.     5.04  78.38 101.5   40.24  70.67  61.14  34.26
  81.59 134.05 119.69 140.25  21.37  91.09 143.43 -39.1   84.91 110.59]

Asymmetric scale: 0.7236470588235294, zero: 55.0
[255   0  55  62 163 195 111 153 139 102 168 240 220 249  85 181 253   1
 172 208]

Symmetric scale: 1.137244094488189
[127 -35   0   4  69  89  35  62  54  30  72 118 105 123  19  80 126 -34
  75  97]


In [3]:
# Dequantize the parameters back to 32 bits
params_deq_asymmetric = asymmetric_dequantize(asymmetric_q, asymmetric_scale, asymmetric_zero)
params_deq_symmetric = symmetric_dequantize(symmetric_q, symmetric_scale)

print(f'Original:')
print(np.round(params, 2))
print('')
print(f'Dequantize Asymmetric:')
print(np.round(params_deq_asymmetric,2))
print('')
print(f'Dequantize Symmetric:')
print(np.round(params_deq_symmetric, 2))

Original:
[144.43 -40.1    0.     5.04  78.38 101.5   40.24  70.67  61.14  34.26
  81.59 134.05 119.69 140.25  21.37  91.09 143.43 -39.1   84.91 110.59]

Dequantize Asymmetric:
[144.73 -39.8    0.     5.07  78.15 101.31  40.52  70.92  60.79  34.01
  81.77 133.87 119.4  140.39  21.71  91.18 143.28 -39.08  84.67 110.72]

Dequantize Symmetric:
[144.43 -39.8    0.     4.55  78.47 101.21  39.8   70.51  61.41  34.12
  81.88 134.19 119.41 139.88  21.61  90.98 143.29 -38.67  85.29 110.31]


In [5]:
# Calculate the quantization error
print(f'{"Asymmetric error: ":>20}{np.round(quantization_error(params, params_deq_asymmetric), 2)}')
print(f'{"Symmetric error: ":>20}{np.round(quantization_error(params, params_deq_symmetric), 2)}')

  Asymmetric error: 0.05
   Symmetric error: 0.08
